<a href="https://colab.research.google.com/github/diana1915/spam_project/blob/main/VK_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# EDA

In [29]:
df = pd.read_csv('/content/drive/MyDrive/VK/train_spam.csv')
test_df = pd.read_csv('/content/drive/MyDrive/VK/test_spam.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16278 entries, 0 to 16277
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_type  16278 non-null  object
 1   text       16278 non-null  object
dtypes: object(2)
memory usage: 254.5+ KB


In [31]:
df.isna().sum() # пропущенных значений нет

text_type    0
text         0
dtype: int64

In [32]:
df.text_type.value_counts() # датасет несбалансированный

text_type
ham     11469
spam     4809
Name: count, dtype: int64

In [33]:
df.drop_duplicates(inplace=True) # удалим дубликаты

In [34]:
df['spam']=df['text_type'].apply(lambda x: 1 if x == 'spam' else 0)
df.head(5)

,text_type,text,spam
0,ham,make sure alex knows his birthday is over in f...,0
1,ham,a resume for john lavorato thanks vince i will...,0
2,spam,plzz visit my website moviesgodml to get all m...,1
3,spam,urgent your mobile number has been awarded wit...,1
4,ham,overview of hr associates analyst project per ...,0


# Models

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['spam'], test_size=0.25, stratify=df['spam'], random_state =42)

In [36]:
len(X_train), len(X_test)

(12200, 4067)

In [37]:
#CounterVectorizer Convert the text into matrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [38]:
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [39]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [40]:
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, classification_report, auc

In [41]:
pred = clf.predict(X_test)
roc_auc_score(y_test, pred)

0.91662790056131

In [51]:
# Запишем итоговый датасет
predict_test = clf.predict(test_df['text'])
result = pd.DataFrame({'score': None, 'label': predict_test, 'text': test_df['text']})
result['score'] = result['label'].apply(lambda x: 'spam' if x == 1 else 'ham')
result.drop(columns='label', inplace=True)
result.to_csv('result.csv', index=False)

In [52]:
result

,score,text
0,ham,j jim whitehead ejw cse ucsc edu writes j you ...
1,ham,original message from bitbitch magnesium net p...
2,ham,java for managers vince durasoft who just taug...
3,ham,there is a youtuber name saiman says
4,spam,underpriced issue with high return on equity t...
...,...,...
4065,spam,husband to wifetum meri zindagi hoorwifeor kya...
4066,ham,baylor enron case study cindy yes i shall co a...
4067,ham,boring as compared to tp
4068,ham,hellogorgeous hows u my fone was on charge lst...
